In [ ]:
%matplotlib inline

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from sklearn.datasets import fetch_20newsgroups

In [ ]:
newsgroups = fetch_20newsgroups(remove=('headers', 'footers'))

In [ ]:
newsgroups.target_names

In [ ]:
newsgroups.data[0]

In [ ]:
newsgroups.target_names[newsgroups.target[0]]

In [ ]:
origin = newsgroups.target_names[newsgroups.target[0]]
print(f"The post at index 0 first appeared in the '{origin}' group")

In [ ]:
len(newsgroups.data)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
vectorizer = CountVectorizer(stop_words='english')

In [ ]:
tf_matrix = vectorizer.fit_transform(newsgroups.data)

In [ ]:
type(tf_matrix)

In [ ]:
tf_np_matrix = tf_matrix.toarray()

In [ ]:
tf_vector = tf_np_matrix[0]
non_zero_indices = np.flatnonzero(tf_vector)
words = vectorizer.get_feature_names_out()
unique_words = [words[index] for index in non_zero_indices]
data = {'Word': unique_words,
'Count': tf_vector[non_zero_indices]}
df = pd.DataFrame(data).sort_values('Count', ascending=False)
print(f"After stop-word deletion, {df.shape[0]} unique words remain.")

In [ ]:
df.head(10)

In [ ]:
df.Word

In [ ]:
non_zero_indices

In [ ]:
sub_matrix = tf_np_matrix[:, non_zero_indices]

In [ ]:
sub_matrix[0]

In [ ]:
from sklearn.preprocessing import binarize

In [ ]:
binary_matrix = binarize(sub_matrix)

In [ ]:
binary_matrix

In [ ]:
binary_matrix.sum(axis=0).shape

In [ ]:
unique_post_mentions = binary_matrix.sum(axis=0)

In [ ]:
unique_post_mentions

In [ ]:
np_post_mentions = binarize(tf_np_matrix[:,non_zero_indices]).sum(axis=0)
csr_post_mentions = binarize(tf_matrix[:,non_zero_indices]).sum(axis=0)
print(f'NumPy matrix-generated counts:\n {np_post_mentions}\n')
print(f'CSR matrix-generated counts:\n {csr_post_mentions}')

In [ ]:
dataset_size = len(newsgroups.data)
document_frequencies = unique_post_mentions / dataset_size

In [ ]:
data = {'Word': unique_words,
'Count': tf_vector[non_zero_indices],
'Document Frequency': document_frequencies}

In [ ]:
df = pd.DataFrame(data)

In [ ]:
df

In [ ]:
df_common_words = df[df['Document Frequency'] > 0.1]

In [ ]:
df_common_words.head(10)

In [ ]:
df_sorted = df.sort_values(['Count','Document Frequency'], ascending=[False, True])

In [ ]:
df_sorted.head(10)

In [ ]:
inverse_document_frequencies = 1 / document_frequencies

In [ ]:
df['IDF'] = inverse_document_frequencies

In [ ]:
df['Combined'] = df.Count * df.IDF

In [ ]:
df['Combined']

In [ ]:
df_sorted = df.sort_values('Combined', ascending=False)

In [ ]:
df['Combined'] = df.Count * np.log10(df.IDF)

In [ ]:
df_sorted = df.sort_values('Combined', ascending=False)

In [ ]:
df_sorted.head(10)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
tfidf_vectorizer = TfidfVectorizer(stop_words='english')

In [ ]:
tfidf_matrix = tfidf_vectorizer.fit_transform(newsgroups.data)

In [ ]:
tfidf_matrix[0]

In [ ]:
assert tfidf_matrix.shape == tf_matrix.shape

In [ ]:
assert np.all(tfidf_vectorizer.get_feature_names_out() == words)

In [ ]:
tfidf_np_matrix = tfidf_matrix.toarray()

In [ ]:
tfidf_vector = tfidf_np_matrix[0]
tfidf_non_zero_indices = np.flatnonzero(tfidf_vector)
assert np.array_equal(tfidf_non_zero_indices,
non_zero_indices)

In [ ]:
df['TFIDF'] = tfidf_vector[non_zero_indices]

In [ ]:
df

In [ ]:
df_sorted_old = df.sort_values('Combined', ascending=False)
df_sorted_new = df.sort_values('TFIDF', ascending=False)
assert np.array_equal(df_sorted_old['Word'].values, df_sorted_new['Word'].values)

In [ ]:
df_sorted_new.head(10)

In [ ]:
np.linalg.norm(df.TFIDF.values)

In [ ]:
tfidf_np_matrix @ tfidf_np_matrix[0]

In [ ]:
cosine_similarities = _

In [ ]:
np.argsort(cosine_similarities)[-2]

In [ ]:
most_similar_index = np.argsort(cosine_similarities)[-2]

In [ ]:
similarity = cosine_similarities[most_similar_index]

In [ ]:
most_similar_post = newsgroups.data[most_similar_index]
print(f"The following post has a cosine similarity of {similarity:.2f} "
"with newsgroups.data[0]:\n")
print(most_similar_post)

In [ ]:
from sklearn.decomposition import TruncatedSVD

In [ ]:
truncated_svd = TruncatedSVD(n_components=100)

In [ ]:
np.random.seed(0)
shrunk_matrix = truncated_svd.fit_transform(tfidf_matrix)

In [ ]:
shrunk_matrix.shape

In [ ]:
magnitude = np.linalg.norm(shrunk_matrix[0])

In [ ]:
magnitude

In [ ]:
from sklearn.preprocessing import normalize

In [ ]:
shrunk_norm_matrix = normalize(shrunk_matrix)

In [ ]:
shrunk_norm_matrix.shape

In [ ]:
shrunk_norm_matrix @ shrunk_norm_matrix.T

In [ ]:
cosine_similarity_matrix = _

In [ ]:
cosine_similarity_matrix.shape

In [ ]:
np.random.seed(1)
index1 = np.random.randint(dataset_size)

In [ ]:
index2 = np.argsort(cosine_similarity_matrix[index1])[-2]
similarity = cosine_similarity_matrix[index1][index2]
print(f"The posts at indices {index1} and {index2} share a cosine " f"similarity of {similarity:.2f}")

In [ ]:
print(newsgroups.data[index1].replace('\n\n', '\n'))

In [ ]:
print(newsgroups.data[index2].replace('\n\n', '\n'))